In [3]:
import numpy as np
import pandas as pd
import os
#import matplotlib.pyplot as plt
import cv2
#import csv
#from keras.utils import to_categorical
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout, BatchNormalization, Activation
from keras.models import Sequential
from keras.optimizers import rmsprop
#import random
#from PIL import Image
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers

In [6]:
np.random.seed(1)


# Processing training data
# -> appending images in a list 'train_images'
# -> appending labels in a list 'train_labels'

train_images = []
train_labels = []
shape = (50, 50)
train_path = 'Training_Set//GTSRB//Final_Training//Images'
for c in range(43):
        sub_dir = train_path + '//' + format(c, '05d')  # subdirectory for class
        for filename in os.listdir(sub_dir):
            if filename.split('.')[1] == 'ppm':
                img = cv2.imread(os.path.join(sub_dir, filename))
                train_labels.append(c)
                # Resize all images to a specific shape
                img = cv2.resize(img, shape)

                train_images.append(img)
# Converting train_images to array
train_images = np.array(train_images)
# Converting labels into One Hot encoded sparse matrix
train_labels = pd.get_dummies(train_labels).values

# Splitting Training data into train and validation dataset
x_train, x_val, y_train, y_val = train_test_split(train_images, train_labels, random_state=1)

In [7]:
model = Sequential()
model.add(BatchNormalization())
model.add(Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu')) 
model.add(Dropout(0.3)) #Overfitting vermeiden
model.add(Dense(43, activation='softmax'))
model.compile(
loss='categorical_crossentropy',
metrics=['acc'],
optimizer='RMSprop')

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
history = model.fit(x_train, y_train, epochs=50,batch_size=512,validation_data=(x_val,y_val))

Train on 29406 samples, validate on 9803 samples
Epoch 1/50
29406/29406 [==============================] - 5s 165us/step - loss: 0.6051 - acc: 0.8298 - val_loss: 0.6967 - val_acc: 0.8263
Epoch 2/50
29406/29406 [==============================] - 5s 165us/step - loss: 0.5431 - acc: 0.8432 - val_loss: 0.3719 - val_acc: 0.8880
Epoch 3/50
29406/29406 [==============================] - 5s 165us/step - loss: 0.5137 - acc: 0.8603 - val_loss: 0.3813 - val_acc: 0.8888
Epoch 4/50
29406/29406 [==============================] - 5s 164us/step - loss: 0.5318 - acc: 0.8669 - val_loss: 0.7565 - val_acc: 0.8239
Epoch 5/50
29406/29406 [==============================] - 5s 165us/step - loss: 0.4755 - acc: 0.8704 - val_loss: 0.4353 - val_acc: 0.8910
Epoch 6/50
29406/29406 [==============================] - 5s 165us/step - loss: 0.4695 - acc: 0.8738 - val_loss: 0.4612 - val_acc: 0.8568
Epoch 7/50
29406/29406 [==============================] - 5s 164us/step - loss: 0.4052 - acc: 0.8905 - val_loss: 0.2303 - v